# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [2]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [3]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [4]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [5]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [6]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [11]:
# ... ENTER YOUR CODE HERE ...
# the function outputs 1 if the requirement for LU decomposition is not satisfied, it outputs 0 if the requirement is satisfied

def check_leading_minors(a):
    not_satisfied=0
    N=a.shape[0]
    for j in range(N):
        dim=np.linalg.matrix_rank(a[:j+1,:j+1])
        if dim<j+1:
            not_satisfied=1
    return not_satisfied

print("check a:", check_leading_minors(a))
print("check a1:", check_leading_minors(a1))

check a: 0
check a1: 1


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [14]:
# ... ENTER YOUR CODE HERE ...
def diy_lu_pivot(a):
    N = a.shape[0]
    U = a.copy()
    L = np.eye(N)
    P_cum = np.eye(N)
    for j in range(N - 1):
        P_cur=np.eye(N)
        pivot=np.argmax(np.abs(U[j:,j]))
        P_cur[[j,pivot+j]]=P_cur[[pivot+j,j]]
        P_cum=np.dot(P_cur,P_cum)

        U=np.dot(P_cur,U)
        lam = np.eye(N)
        gamma = U[j+1:, j] / U[j, j]
        lam[j+1:, j] = -gamma
        #@ is the same as np.dot
        U = np.dot(lam,U)

        L=np.dot(P_cur,L)
        L = np.dot(lam,L)

    return np.dot(P_cum,np.linalg.inv(L)), U, np.linalg.inv(P_cum)
L,U,P=diy_lu_pivot(a)
print('L\n', L, '\n U', U, '\n P\n', P)
L1,U1,P1=diy_lu_pivot(a1)
print('L1\n', L1, '\n U1', U1, '\n P1', P1)


L
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]] 
 U [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  1.110e-16 -2.317e-17 -1.999e-19 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.431e-16  6.463e-18  1.577e-19  0.000e+00  6.730e-06]] 
 P
 [[ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [-0. -0. -0. -0.  1. -0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.]]
L1
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0

In [15]:
def reconstruct(p,l,u):
    return np.dot(p,np.dot(l,u))
a_rec=reconstruct(P,L,U)
a1_rec=reconstruct(P1,L1,U1)
#the very small differences are caused by numerical reason
print("compare a and a_rec\n", a-a_rec, "\n")
print("compare a1 and a1_rec\n", a1-a1_rec, "\n")

compare a and a_rec
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17 -2.776e-16 -2.776e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  1.665e-16  1.665e-16 -5.551e-17  0.000e+00]] 

compare a1 and a1_rec
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 4.441e-16  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  1.665e-16  1.665e-16 -5.551e-17  0.000e+00]] 

